In [10]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys


# Scrapping avec Selenium et Safari

In [ ]:
url = "https://www.carrefour.fr/r/fruits-et-legumes/legumes"
num_products_to_scrape = 50  


# Initialiser le navigateur Chrome
#chrome_path = "/Users/victor/dev/chromedriver/ChromeDriver 114.0.5735.90/chromedriver" 
#chrome_service = ChromeService(chrome_path)
#driver = webdriver.Chrome(service=chrome_service)
options = webdriver.SafariOptions()
driver = webdriver.Safari(options=options)

#  à l'URL
driver.get(url)

time.sleep(5)

scraped_product_names = set()

driver.execute_script("""
    var rejectButton = document.getElementById("onetrust-reject-all-handler");
    if (rejectButton) {
        rejectButton.click();
    } else {
        console.log("Le bouton n'a pas été trouvé.");
    }
""")

data = []

while len(data) < num_products_to_scrape:
    product_cards = driver.find_elements(By.CLASS_NAME, "product-grid-item")

    # Parcourir les cartes de produits
    for product_card in product_cards:
        try:
            # Récupérer le nom du produit
            name = product_card.find_element(By.CLASS_NAME, "product-card-title__text").text if product_card.find_elements(By.CLASS_NAME, "product-card-title__text") else None

            # Vérifier si le produit a déjà été récupéré
            if name not in scraped_product_names:
                # lien de l'image
                #image_link = product_card.find_element(By.CLASS_NAME, "image image--hidden product-card-image__image ls-is-cached image--loaded").get_attribute("src") if product_card.find_elements(By.CLASS_NAME, "main-horizontal__image") else None
                image_element = driver.find_element(By.CLASS_NAME, 'product-card-image__image')
                image_link = image_element.get_attribute('src')
                # origine du produit
                origin = product_card.find_element(By.CLASS_NAME, "ds-product-card-refonte__origin-text").text if product_card.find_elements(By.CLASS_NAME, "ds-product-card-refonte__origin-text") else None

                #prix
                price = product_card.find_element(By.CLASS_NAME, "product-price__amount-value").text if product_card.find_elements(By.CLASS_NAME, "product-price__amount-value") else None

                #prix à l'unité
                price_per_unit = product_card.find_element(By.CLASS_NAME, "ds-product-card-refonte__perunitlabel").text if product_card.find_elements(By.CLASS_NAME, "ds-product-card-refonte__perunitlabel") else None

                # Ajouter les données à la liste
                data.append({
                    'Nom': name,
                    'Image': image_link,
                    'Origine': origin,
                    'Prix': price,
                    'Prix à l\'unité': price_per_unit
                })

                # Ajouter le nom du produit à l'ensemble des produits déjà récupérés
                scraped_product_names.add(name)
        except Exception as e:
            print(f"Erreur lors du traitement d'une carte de produit : {e}")
    # Ajouter le scroll en bas de la page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(1)
    # On vérifie si le bouton "Voir plus" est présent
    see_more_button = driver.find_elements(By.CLASS_NAME, "pl-button-deprecated--primary")


    if see_more_button:
        print("Il y a un bouton")

        see_more_button.click()
        time.sleep(2)
    else:
        print("Il n'y a plus de bouton")
        break

driver.quit()

df = pd.DataFrame(data)

print(df)

df.to_csv('fruits_data.csv', index=False)